In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib as mpl
import os
mpl.rcParams["lines.linewidth"] = 0.96
%matplotlib qt

from ppg_fiducials import *

import warnings
warnings.filterwarnings("ignore")

In [3]:
import glob

# HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM_mod/'
HOM_path = '/Volumes/Untitled/Hom3ostasis/HOM_mod/HOM_mod/'
data_path = '/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])
len(sub_names)

39

## Full 24-h recording

In [5]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]

    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()

    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    ppg = ppg_data[2:].to_numpy().squeeze()
    t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
    ppg_df = pd.DataFrame(ppg, index=t_ppg, columns=["ppg"])

    output_GGIR_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/" + sub_ID + "/output_GGIR/output_input_GGIR/results/QC/"

    output_GGIR = pd.read_csv(output_GGIR_path + "part4_nightsummary_sleep_full.csv")

    if output_GGIR["sleeponset_ts"].iloc[0][0] == '0':
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date()+ pd.Timedelta("1d")) + " " + output_GGIR["sleeponset_ts"].iloc[0])
    else: 
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date())  + " " + output_GGIR["sleeponset_ts"].iloc[0])

    wake_onset = pd.to_datetime(str(ppg_df.index[0].date() + pd.Timedelta("1d")) + " " + output_GGIR["wakeup_ts"].iloc[0])

    ix_sleep_onset = ppg_df.index.get_loc(sleep_onset)
    
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1

    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ppg_datetime = ppg_df.index
    I_foot_datetime = ppg_datetime[I_foot]

    I_foot_basic_datetime = I_foot_datetime[:-1][y_basic == 1]
    I_foot_diag_datetime = I_foot_datetime[:-1][y_diagnostic == 2]

    ppg = ppg_df["ppg"].to_numpy()
    ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    # --- Diagnostic fiducials ---
    I_peak_diag = I_peak[y_diagnostic == 2]
    start_diag = I_foot[:-1][y_diagnostic == 2]
    end_diag = I_foot[1:][y_diagnostic == 2]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot)) # TODO: check if 64 makes sense since i'm interpolating to 100 samples -- I think I'm not using this value anywhere for the fiducials point calculation
    dn_diag = np.array(dn)[y_diagnostic == 2]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 2].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 2].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 2].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 2].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 2].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"start_datetime": I_foot_diag_datetime, "onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag.drop(columns = "start_datetime") < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    plt.figure()
    plt.plot(ppg)
    plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=9)
    plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=9)
    for j in range(y_basic.shape[0]):
        if y_basic[j] == 1:
            plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')

    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features_diag = pd.DataFrame({"Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features_diag.index = pd.to_datetime(fiducials_diag["start_datetime"])
    features_diag["sleep"] = 0
    features_diag.loc[sleep_onset:wake_onset, "sleep"] = 1

    # --- Basic fiducials ---
    I_peak_basic = I_peak[y_basic == 1]
    start_basic = I_foot[:-1][y_basic == 1]
    end_basic = I_foot[1:][y_basic == 1]
    basic_pulses = np.sort(np.concatenate((start_basic, end_basic)))

    u_basic = vpg_fp["u"][y_basic == 1].values.astype(int)

    fiducials_basic = pd.DataFrame({"start_datetime": I_foot_basic_datetime, "onset": basic_pulses[::2], "offset": basic_pulses[1::2]-1, "sp": I_peak_basic, "u": u_basic})
    fiducials_basic[fiducials_basic.drop(columns = "start_datetime") < -5000000] = np.nan
    fiducials_basic.dropna(inplace=True)
    fiducials_basic = fiducials_basic.astype(int)

    Tpi = fiducials_basic["offset"] - fiducials_basic["onset"]
    Tsp = fiducials_basic["sp"] - fiducials_basic["onset"]
    Asp = ppg[fiducials_basic["sp"]] - ppg[fiducials_basic["onset"]]
    AUCpi = np.zeros(len(fiducials_basic))
    for i in range(len(fiducials_basic)):
        AUCpi[i] = np.trapz(ppg[fiducials_basic["onset"].iloc[i]:fiducials_basic["offset"].iloc[i]])
    TspTpi = Tsp/Tpi
    TspAsp = Tsp/Asp

    features_basic = pd.DataFrame({"Tsp": Tsp, "Tpi": Tpi, "Asp": Asp, "AUCpi": AUCpi, "Tsp/Tpi": TspTpi, "Tsp/Asp": TspAsp})
    features_basic.index = pd.to_datetime(fiducials_basic["start_datetime"])
    features_basic["sleep"] = 0
    features_basic.loc[sleep_onset:wake_onset, "sleep"] = 1

    # features_diag.to_csv(data_path + sub_ID + '/features_diag_ALL.csv', index = True)
    # fiducials_diag.to_csv(data_path + sub_ID + '/fiducials_diag_ALL.csv', index = True)

    # features_basic.to_csv(data_path + sub_ID + '/features_basic_ALL.csv', index = True)
    # fiducials_basic.to_csv(data_path + sub_ID + '/fiducials_basic_ALL.csv', index = True)

10_AC


: 

In [76]:
sleep_onset, wake_onset

(Timestamp('2020-07-23 22:38:10'), Timestamp('2020-07-24 07:00:15'))

In [79]:
features_basic.shape, features_diag.shape

features_basic_2min30sec = features_basic.resample("2min30s").median()
features_basic_2min30sec["Tsp"].plot()

<Axes: xlabel='start_datetime'>

In [68]:
features_2min30sec = features.resample("2.5 min").median()
features_2min30sec["sleep"] = 0
features_2min30sec.loc[sleep_onset:wake_onset, "sleep"] = 1

In [73]:
plt.figure(figsize=(10, 6))
plt.plot(features_2min30sec["Tsys"])
plt.plot(features_2min30sec["sleep"]*100)

In [62]:
features_hourly = features.resample("2.5 min").median().resample("1 H").mean()
features_hourly["N_diagnostic"] = features.resample("1 H").count()["Tsys"]

In [65]:
features_hourly

,Tsys,Tdia,Tsp,Tdp,dT,Asp,Adn,Adp,AUCpi,AUCsys,...,IPA,Tsp/Asp,Asp/dT,Asp/(Tpi-Tsp),Rslope,Tu,Tv,Ta,Tb,N_diagnostic
start_datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-23 09:00:00,45.500000,53.500000,30.392857,51.321429,20.571429,79.405862,66.624809,65.727908,122.527469,305.012101,...,-0.280451,0.403673,4.188535,1.165229,1.650960,12.035714,55.571429,11.035714,34.500000,141
2020-07-23 10:00:00,49.375000,49.625000,30.406250,50.968750,20.000000,93.764561,71.487614,74.112146,351.965601,493.569927,...,-0.727461,0.517391,4.844920,1.361536,1.943904,13.593750,52.968750,12.593750,34.000000,68
2020-07-23 11:00:00,48.264706,50.735294,31.029412,51.941176,19.705882,58.353544,45.751988,46.935671,129.800867,293.493782,...,-0.540527,0.594634,3.140219,0.886227,1.217974,14.264706,54.176471,13.264706,33.500000,43
2020-07-23 12:00:00,47.978261,51.021739,31.869565,50.673913,17.804348,80.996069,64.284513,72.286146,423.292491,501.247399,...,0.426522,0.507622,5.041500,1.254572,1.666986,13.260870,54.804348,12.260870,35.282609,92
2020-07-23 13:00:00,45.050000,53.950000,29.425000,51.200000,20.475000,56.679513,47.593561,49.499490,36.886516,157.361330,...,-0.280603,0.612730,2.967199,0.827977,1.242150,11.650000,54.625000,10.650000,33.200000,94
2020-07-23 14:00:00,47.565217,51.434783,31.521739,53.434783,21.239130,72.334196,54.838467,57.714967,228.461131,371.838273,...,-0.111590,0.551643,3.633105,1.089020,1.559979,14.108696,53.195652,13.108696,34.239130,102
2020-07-23 15:00:00,48.250000,50.750000,30.575000,54.625000,22.850000,64.725402,53.748992,54.606376,195.087261,293.796716,...,-0.279210,0.538754,3.109824,0.961955,1.413934,12.225000,54.175000,11.225000,32.325000,91
2020-07-23 16:00:00,48.229167,50.770833,31.166667,49.854167,18.166667,38.521700,30.639036,32.907807,175.266448,237.254070,...,-0.386979,0.850388,2.445571,0.579011,0.806108,12.541667,56.479167,11.541667,34.250000,126
2020-07-23 17:00:00,45.312500,53.687500,28.750000,51.312500,22.604167,88.911702,65.393026,65.293369,165.984543,498.414136,...,-0.573540,0.378224,3.934184,1.256656,1.972428,11.562500,49.020833,10.562500,34.125000,185


In [60]:
features.resample("1H").count()

,Tsys,Tdia,Tsp,Tdp,dT,Asp,Adn,Adp,AUCpi,AUCsys,...,Adp/Asp,IPA,Tsp/Asp,Asp/dT,Asp/(Tpi-Tsp),Rslope,Tu,Tv,Ta,Tb
start_datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-23 09:00:00,141,141,141,141,141,141,141,141,141,141,...,141,141,141,141,141,141,141,141,141,141
2020-07-23 10:00:00,68,68,68,68,68,68,68,68,68,68,...,68,68,68,68,68,68,68,68,68,68
2020-07-23 11:00:00,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
2020-07-23 12:00:00,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92
2020-07-23 13:00:00,94,94,94,94,94,94,94,94,94,94,...,94,94,94,94,94,94,94,94,94,94
2020-07-23 14:00:00,102,102,102,102,102,102,102,102,102,102,...,102,102,102,102,102,102,102,102,102,102
2020-07-23 15:00:00,91,91,91,91,91,91,91,91,91,91,...,91,91,91,91,91,91,91,91,91,91
2020-07-23 16:00:00,126,126,126,126,126,126,126,126,126,126,...,126,126,126,126,126,126,126,126,126,126
2020-07-23 17:00:00,185,185,185,185,185,185,185,185,185,185,...,185,185,185,185,185,185,185,185,185,185


## DAY1

In [38]:
fs = 64
sub_names = ["21_CK"]
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]

    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()

    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    ppg = ppg_data[2:].to_numpy().squeeze()
    t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
    ppg_df = pd.DataFrame(ppg, index=t_ppg, columns=["ppg"])

    output_GGIR_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/" + sub_ID + "/output_GGIR/output_input_GGIR/results/QC/"

    output_GGIR = pd.read_csv(output_GGIR_path + "part4_nightsummary_sleep_full.csv")

    if output_GGIR["sleeponset_ts"].iloc[0][0] == '0':
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date()+ pd.Timedelta("1d")) + " " + output_GGIR["sleeponset_ts"].iloc[0])
    else: 
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date())  + " " + output_GGIR["sleeponset_ts"].iloc[0])

    wake_onset = pd.to_datetime(str(ppg_df.index[0].date() + pd.Timedelta("1d")) + " " + output_GGIR["wakeup_ts"].iloc[0])

    ppg_day1 = ppg_df.loc[ppg_df.index[0]:sleep_onset]

    ix_sleep_onset = ppg_df.index.get_loc(sleep_onset)
    
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1

    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ix_stop = np.argmin(np.abs(ix_sleep_onset - I_foot))

    I_foot = I_foot[:ix_stop+1]
    I_peak = I_peak[:ix_stop+1]
    ppg_datetime = ppg_day1.index
    I_foot_datetime = ppg_datetime[I_foot]

    y_basic = y_basic[:ix_stop]
    y_diagnostic = y_diagnostic[:ix_stop]
    I_foot_diag_datetime = I_foot_datetime[:-1][y_diagnostic == 2]


    ppg = ppg_day1["ppg"].to_numpy()
    ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    I_peak_diag = I_peak[y_diagnostic == 2]
    start_diag = I_foot[:-1][y_diagnostic == 2]
    end_diag = I_foot[1:][y_diagnostic == 2]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot)) # TODO: check if 64 makes sense since i'm interpolating to 100 samples
    dn_diag = np.array(dn)[y_diagnostic == 2]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 2].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 2].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 2].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 2].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 2].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"start_datetime": I_foot_diag_datetime, "onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag.drop(columns = "start_datetime") < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=9)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=9)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 2:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')

    # break
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features = pd.DataFrame({"Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.index = pd.to_datetime(fiducials_diag["start_datetime"])
    break
    # features.to_csv(data_path + sub_ID + '/' + '/features_day1.csv', index = False)
    # fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials_day1.csv', index = False)

21_CK


IndexError: index 3044238 is out of bounds for axis 0 with size 3044225

In [28]:
features

,Tsys,Tdia,Tsp,Tdp,dT,Asp,Adn,Adp,AUCpi,AUCsys,...,Adp/Asp,IPA,Tsp/Asp,Asp/dT,Asp/(Tpi-Tsp),Rslope,Tu,Tv,Ta,Tb
start_datetime,,,,,,,,,,,,,,,,,,,,,
2019-11-19 10:18:31.312500,48,51,36,52,16,55.462727,50.099697,49.720000,260.898737,122.650657,...,0.896458,0.968175,0.649085,3.466420,0.880361,1.155473,18,64,17,41
2019-11-19 10:18:42.187500,56,43,33,61,28,42.720000,36.165455,36.509091,-40.510000,85.047273,...,0.854614,-1.584413,0.772472,1.525714,0.647273,0.762857,21,64,20,37
2019-11-19 10:19:07.687500,44,55,33,50,17,80.216667,76.024444,76.768182,386.410455,117.827980,...,0.957010,2.024186,0.411386,4.718627,1.215404,1.823106,14,64,13,37
2019-11-19 10:19:10.562500,45,54,34,50,16,60.897475,59.003636,59.317374,866.832172,422.259798,...,0.974053,0.982277,0.558315,3.806092,0.936884,1.353277,15,64,14,37
2019-11-19 10:19:57.875000,53,46,33,56,23,81.510000,76.127576,76.160000,1059.955758,626.171364,...,0.934364,0.637846,0.404858,3.543913,1.235000,1.537925,19,64,18,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-19 22:45:08.734375,52,47,31,66,35,49.753333,29.106667,30.330000,-374.113485,103.821970,...,0.609607,-4.595596,0.623074,1.421524,0.731667,0.956795,12,41,11,41
2019-11-19 22:45:09.359375,48,51,29,57,28,51.523333,39.128788,40.900909,169.816414,53.901162,...,0.793833,2.033726,0.562852,1.840119,0.736048,1.073403,14,37,13,37
2019-11-19 22:49:36.562500,37,62,24,45,21,122.866667,96.071111,93.040000,3553.480556,1932.072222,...,0.757244,0.802658,0.195334,5.850794,1.638222,3.320721,12,30,11,30


In [31]:
features.resample("2.5 min").median()

,Tsys,Tdia,Tsp,Tdp,dT,Asp,Adn,Adp,AUCpi,AUCsys,...,Adp/Asp,IPA,Tsp/Asp,Asp/dT,Asp/(Tpi-Tsp),Rslope,Tu,Tv,Ta,Tb
start_datetime,,,,,,,,,,,,,,,,,,,,,
2019-11-19 10:17:30,48.0,51.0,33.0,52.0,17.0,60.897475,59.003636,59.317374,386.410455,122.650657,...,0.934364,0.968175,0.558315,3.543913,0.936884,1.353277,18.0,64.0,17.0,37.0
2019-11-19 10:20:00,44.5,54.5,34.0,51.0,19.0,66.702121,54.620758,54.299343,238.540707,230.401919,...,0.901245,0.890160,0.501565,3.334679,1.062948,1.452935,16.0,64.0,15.0,37.0
2019-11-19 10:22:30,47.0,52.0,34.0,52.0,17.0,80.184545,72.299697,72.118182,49.791465,218.224444,...,0.899403,-0.490905,0.429620,5.224505,1.252884,1.647184,18.0,61.0,17.0,37.0
2019-11-19 10:25:00,44.5,54.5,30.5,47.0,17.0,41.143586,35.644343,39.043081,119.565480,150.735000,...,0.897830,-0.068977,0.689980,2.256603,0.593829,0.950714,10.0,62.5,9.0,33.5
2019-11-19 10:27:30,45.5,53.5,31.5,53.5,22.0,68.699192,63.323485,63.758889,-494.726061,-173.410657,...,0.929832,0.416341,0.484070,3.361139,1.032015,1.512434,17.5,62.5,16.5,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-19 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-19 23:02:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-19 23:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
fiducials_diag["start_datetime"] = pd.to_datetime(fiducials_diag["start_datetime"])
fiducials_diag

,start_datetime,onset,offset,sp,dn,dp,u,v,a,b
0,2019-11-19 10:18:31.312500,700,799,736,748,752,718,764,717,741
1,2019-11-19 10:18:42.187500,2200,2299,2233,2256,2261,2221,2264,2220,2237
3,2019-11-19 10:19:07.687500,5700,5799,5733,5744,5750,5714,5764,5713,5737
4,2019-11-19 10:19:10.562500,6100,6199,6134,6145,6150,6115,6164,6114,6137
5,2019-11-19 10:19:57.875000,12700,12799,12733,12753,12756,12719,12764,12718,12746
...,...,...,...,...,...,...,...,...,...,...
795,2019-11-19 22:45:08.734375,4978900,4978999,4978931,4978952,4978966,4978912,4978941,4978911,4978941
796,2019-11-19 22:45:09.359375,4979000,4979099,4979029,4979048,4979057,4979014,4979037,4979013,4979037
797,2019-11-19 22:49:36.562500,5007900,5007999,5007924,5007937,5007945,5007912,5007930,5007911,5007930
798,2019-11-19 22:58:29.109375,5075900,5075999,5075939,5075959,5075968,5075923,5075953,5075922,5075953


In [10]:
np.sum(y_diagnostic == 2)

801

## Night

In [9]:
t_ppg[-1]

Timestamp('2020-08-06 00:07:16.593750')

In [11]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]

    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()

    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    ppg = ppg_data[2:].to_numpy().squeeze()
    t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
    ppg_df = pd.DataFrame(ppg, index=t_ppg, columns=["ppg"])

    output_GGIR_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/" + sub_ID + "/output_GGIR/output_input_GGIR/results/QC/"

    output_GGIR = pd.read_csv(output_GGIR_path + "part4_nightsummary_sleep_full.csv")

    if output_GGIR["sleeponset_ts"].iloc[0][0] == '0':
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date()+ pd.Timedelta("1d")) + " " + output_GGIR["sleeponset_ts"].iloc[0])
    else: 
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date())  + " " + output_GGIR["sleeponset_ts"].iloc[0])

    wake_onset = pd.to_datetime(str(ppg_df.index[0].date() + pd.Timedelta("1d")) + " " + output_GGIR["wakeup_ts"].iloc[0])

    if sub_ID == "22_CM":
        wake_onset = t_ppg[-1]

    ppg_night = ppg_df.loc[wake_onset:sleep_onset]

    ix_sleep_onset = ppg_df.index.get_loc(sleep_onset)
    ix_wake_onset = ppg_df.index.get_loc(wake_onset)
    
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1

    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ix_start = np.argmin(np.abs(ix_sleep_onset - I_foot))
    ix_stop = np.argmin(np.abs(ix_wake_onset - I_foot))

    I_foot = I_foot[ix_start:ix_stop+1]
    I_peak = I_peak[ix_start:ix_stop+1]

    y_basic = y_basic[ix_start:ix_stop]
    y_diagnostic = y_diagnostic[ix_start:ix_stop]

    
    # ppg = ppg_night.values

    ppg_pulses = [ppg_df["ppg"].values[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    I_peak_diag = I_peak[y_diagnostic == 2]
    start_diag = I_foot[:-1][y_diagnostic == 2]
    end_diag = I_foot[1:][y_diagnostic == 2]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 2]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 2].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 2].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 2].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 2].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 2].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=9)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=9)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 2:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')

    # break
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features = pd.DataFrame({"Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(data_path + sub_ID + '/' + '/features_night.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials_night.csv', index = False)

10_AC
11_EC
12_FN


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


13_MS


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


14_AD
15_ER


...

/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


8_MB


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


9_BC


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


In [12]:
ppg_df["ppg"].values

array([-0.  , -0.  , -0.  , ..., 32.36, 15.58, -4.64])

In [50]:
ix_start = np.argmin(np.abs(ix_sleep_onset - I_foot))
ix_stop = np.argmin(np.abs(ix_wake_onset - I_foot))

In [54]:
ix_start, ix_stop

(52735, 82229)

In [5]:
I_foot = I_foot[ix_start:ix_stop]
I_peak = I_peak[ix_start:ix_stop]

y_basic = y_basic[ix_start:ix_stop]
y_diagnostic = y_diagnostic[ix_start:ix_stop]

plt.figure()
plt.plot(ppg_df.values)
plt.plot(I_peak, ppg_df.values[I_peak], 'k*', markersize=6)
for j in range(y_diagnostic.shape[0]):
    if y_diagnostic[j] == 2:
        plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg_df.values[np.arange(I_foot[j],I_foot[j+1])], 'r')

# DAY 2

In [3]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    if sub_ID == "22_CM":
        continue
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]

    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()

    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    ppg = ppg_data[2:].to_numpy().squeeze()
    t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
    ppg_df = pd.DataFrame(ppg, index=t_ppg, columns=["ppg"])

    output_GGIR_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/" + sub_ID + "/output_GGIR/output_input_GGIR/results/QC/"

    output_GGIR = pd.read_csv(output_GGIR_path + "part4_nightsummary_sleep_full.csv")

    if output_GGIR["sleeponset_ts"].iloc[0][0] == '0':
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date()+ pd.Timedelta("1d")) + " " + output_GGIR["sleeponset_ts"].iloc[0])
    else: 
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date())  + " " + output_GGIR["sleeponset_ts"].iloc[0])

    wake_onset = pd.to_datetime(str(ppg_df.index[0].date() + pd.Timedelta("1d")) + " " + output_GGIR["wakeup_ts"].iloc[0])

    ppg_day1 = ppg_df.loc[ppg_df.index[0]:sleep_onset]

    ix_wake_onset = ppg_df.index.get_loc(wake_onset)
    
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1

    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ix_start = np.argmin(np.abs(ix_wake_onset - I_foot))

    I_foot = I_foot[ix_start:]
    I_peak = I_peak[ix_start:]

    y_basic = y_basic[ix_start:]
    y_diagnostic = y_diagnostic[ix_start:]

    ppg_pulses = [ppg_df["ppg"].values[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    I_peak_diag = I_peak[y_diagnostic == 2]
    start_diag = I_foot[:-1][y_diagnostic == 2]
    end_diag = I_foot[1:][y_diagnostic == 2]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 2]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 2].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 2].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 2].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 2].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 2].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=9)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=9)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 2:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')

    # break
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features = pd.DataFrame({"Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(data_path + sub_ID + '/' + '/features_day2.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials_day2.csv', index = False)

10_AC


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


11_EC


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


12_FN


...

/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


7_GM


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


8_MB
9_BC


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


In [18]:
plt.figure()
plt.plot(ppg_df["ppg"].values)

In [7]:
for i, sub_ID in enumerate(sub_names): 

    sub_ID = "22_CM"

    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    
    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()

    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    ppg = ppg_data[2:].to_numpy().squeeze()
    t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
    ppg_df = pd.DataFrame(ppg, index=t_ppg, columns=["ppg"])

    output_GGIR_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/" + sub_ID + "/output_GGIR/output_input_GGIR/results/QC/"

    output_GGIR = pd.read_csv(output_GGIR_path + "part4_nightsummary_sleep_full.csv")

    if output_GGIR["sleeponset_ts"].iloc[0][0] == '0':
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date()+ pd.Timedelta("1d")) + " " + output_GGIR["sleeponset_ts"].iloc[0])
    else: 
        sleep_onset = pd.to_datetime(str(ppg_df.index[0].date())  + " " + output_GGIR["sleeponset_ts"].iloc[0])

    wake_onset = pd.to_datetime(str(ppg_df.index[0].date() + pd.Timedelta("1d")) + " " + output_GGIR["wakeup_ts"].iloc[0])
    print(sleep_onset)
    print(wake_onset)
    print(sleep_onset - ppg_df.index[0])
    print(wake_onset - sleep_onset)
    print(ppg_df.index[-1] - wake_onset)
    break

22_CM
2020-08-05 20:52:30
2020-08-06 23:46:45
0 days 11:51:19
1 days 02:54:15
-1 days +00:20:31.593750


In [8]:
plt.figure()
plt.plot(ppg_df)

## 2:30 min windows

In [3]:
import glob

In [4]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM_mod/'

In [5]:
sub_ID = "12_FN"
print(sub_ID)
ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
# acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
ppg = ppg_data[2:].to_numpy().squeeze()
t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
if I_peak[0] < I_foot[0]:
    I_peak = I_peak[1:]
if I_foot[-1] > I_peak[-1]:
    I_foot = I_foot[:-1]

12_FN


In [19]:
y_basic.shape, I_foot.shape, I_peak.shape

((91300,), (91301,), (91301,))

In [6]:
sampling_rate = 64  # Example sampling rate in Hz
window_duration = 150  # Window duration in seconds

# Calculate the number of data points in each window
window_size = window_duration * sampling_rate

# Calculate the total number of windows possible
total_windows = len(ppg) // window_size

# List to hold each window of data
windows = []
t_windows = []
I_foot_windows = []
I_peak_windows = []
y_basic_windows = []
I_foot = I_foot[:-1]
I_peak = I_peak[:-1]

# Extract each window of data
for i in range(total_windows):
    start_index = i * window_size
    end_index = start_index + window_size + 1
    window = ppg[start_index:end_index]
    t_window = t_ppg[start_index:end_index]
    t_windows.append(t_window)
    I_foot_window = I_foot[(I_foot >= start_index) & (I_foot < end_index)] - start_index
    I_peak_window = I_peak[(I_peak >= start_index) & (I_peak < end_index)] - start_index
    y_basic_window = y_basic[(I_foot >= start_index) & (I_foot < end_index)]
    I_foot_windows.append(I_foot_window)
    I_peak_windows.append(I_peak_window)
    y_basic_windows.append(y_basic_window)
    windows.append(window)

In [7]:
I_foot_windows[winn].shape, I_peak_windows[winn].shape, y_basic_windows[winn].shape

NameError: name 'winn' is not defined

In [62]:
winn = 5
plt.figure()
plt.plot(windows[winn])
plt.plot(I_foot_windows[winn], windows[winn][I_foot_windows[winn]], 'k*', markersize=6)
plt.plot(I_peak_windows[winn], windows[winn][I_peak_windows[winn]], 'r*', markersize=6)
for j in range(len(y_basic_windows[winn])-1):
    if y_basic_windows[winn][j] == 1:
        plt.plot(np.arange(I_foot_windows[winn][j],I_foot_windows[winn][j+1]), windows[winn][np.arange(I_foot_windows[winn][j],I_foot_windows[winn][j+1])], 'r')

In [15]:
for i, win in enumerate(windows[233:]):
    i = 233
    ppg = win
    I_foot = I_foot_windows[i]
    print(I_foot.shape)
    I_peak = I_peak_windows[i]

    y_basic = y_basic_windows[i]
    
    ppg_pulses = [ppg[I_foot[k]:I_foot[k+1]] for k in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    print(I_foot.shape)
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    y_basic = y_basic[:-1]

    I_peak_diag = I_peak[y_basic == 1]
    start_diag = I_foot[:-1][y_basic == 1]
    end_diag = I_foot[:-1][y_basic == 1]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_basic == 1]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_basic == 1].values.astype(int)
    a_diag = apg_fp["a"][y_basic == 1].values.astype(int)
    b_diag = apg_fp["b"][y_basic == 1].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_basic == 1].values.astype(int)
    v_diag = vpg_fp["v"][y_basic == 1].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    plt.figure()
    plt.plot(ppg)
    plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    for j in range(y_basic.shape[0]):
        if y_basic[j] == 1:
            plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')

    break

(132,)
(132,)


In [7]:
import pickle

In [16]:
sampling_rate = 64  # Example sampling rate in Hz
window_duration = 150  # Window duration in seconds

# Calculate the number of data points in each window
window_size = window_duration * sampling_rate

for s, sub_ID in enumerate(sub_names):
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    ppg = ppg_data[2:].to_numpy().squeeze()
    t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    # List to hold each window of data
    windows = []
    t_windows = []
    I_foot_windows = []
    I_peak_windows = []
    y_basic_windows = []
    I_foot = I_foot[:-1]
    I_peak = I_peak[:-1]

    total_windows = len(ppg) // window_size

    # Extract each window of data
    for i in range(total_windows):
        start_index = i * window_size
        end_index = start_index + window_size + 1
        window = ppg[start_index:end_index]
        t_window = t_ppg[start_index:end_index]
        t_windows.append(t_window)
        I_foot_window = I_foot[(I_foot >= start_index) & (I_foot < end_index)] - start_index
        I_peak_window = I_peak[(I_peak >= start_index) & (I_peak < end_index)] - start_index
        y_basic_window = y_basic[(I_foot >= start_index) & (I_foot < end_index)]
        I_foot_windows.append(I_foot_window)
        I_peak_windows.append(I_peak_window)
        y_basic_windows.append(y_basic_window)
        windows.append(window)

    features = {}

    for i, win in enumerate(windows):
        if y_basic_windows[i].sum() < 0.8 * len(y_basic_windows[i]):
            continue

        print(i)
        features["t"] = t_windows[i]
        ppg = win
        I_foot = I_foot_windows[i]
        I_peak = I_peak_windows[i]

        y_basic = y_basic_windows[i]
        
        ppg_pulses = [ppg[I_foot[k]:I_foot[k+1]] for k in range(len(I_foot)-1)]

        # Interpolate each pulse to 100 samples with cubic spline
        ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

        # Reconstruct the overall ppg using the interpolated pulses
        I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
        I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
        try:
            ppg = np.concatenate(ppg_pulses_interp)
        except:
            continue

        y_basic = y_basic[:-1]

        I_peak_diag = I_peak[y_basic == 1]
        start_diag = I_foot[:-1][y_basic == 1]
        end_diag = I_foot[:-1][y_basic == 1]
        diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
        diag_pulses[::2].shape, I_peak_diag.shape

        dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
        dn_diag = np.array(dn)[y_basic == 1]
        apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
        e_diag = apg_fp["e"][y_basic == 1].values.astype(int)
        a_diag = apg_fp["a"][y_basic == 1].values.astype(int)
        b_diag = apg_fp["b"][y_basic == 1].values.astype(int)

        vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
        u_diag = vpg_fp["u"][y_basic == 1].values.astype(int)
        v_diag = vpg_fp["v"][y_basic == 1].values.astype(int)
        dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
        fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
        fiducials_diag[fiducials_diag < -5000000] = np.nan
        fiducials_diag.dropna(inplace=True)
        fiducials_diag = fiducials_diag.astype(int)

        # plt.figure()
        # plt.plot(ppg)
        # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
        # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
        # for j in range(y_basic.shape[0]):
        #     if y_basic[j] == 1:
        #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')

        # break

        # ----- Time Intervals -----
        # Tpi: time between the pulse onset and offset
        # Tsys: time between the pulse onset and dn
        # Tdia: time between the dn and pulse offset
        # Tsp: time between the pulse onset and sp
        # Tdp: time between the pulse onset and dp
        # dT: time between sp and dp
        Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
        Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
        Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
        Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
        Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
        dT = fiducials_diag["dp"] - fiducials_diag["sp"]

        # ---- Amplitudes -----
        # Asp: difference in amplitude between onset and sp
        # Adn: difference in amplitude between onset and dn
        # Adp: difference in amplitude between onset and dp
        Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
        Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
        Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

        # ---- Areas -----
        # AUCpi: area under the whole pulse (between onset and offset)
        # AUCsys: area under pulse wave between onset and dn
        # AUCdia: area under pulse wave between dn and offset
        AUCpi = np.zeros(len(fiducials_diag))
        AUCsys = np.zeros(len(fiducials_diag))
        AUCdia = np.zeros(len(fiducials_diag))
        for l in range(len(fiducials_diag)):
            AUCpi[l] = np.trapz(ppg[fiducials_diag["onset"].iloc[l]:fiducials_diag["offset"].iloc[l]])
            AUCsys[l] = np.trapz(ppg[fiducials_diag["onset"].iloc[l]:fiducials_diag["dn"].iloc[l]])
            AUCdia[l] = np.trapz(ppg[fiducials_diag["dn"].iloc[l]:fiducials_diag["offset"].iloc[l]])

        # ---- Ratios -----
        # TsysTdia: ratio between Tsys and Tdia
        # TspTpi: ratio between Tsp and Tpi
        # AdpAsp: ratio between Adp and Asp --> reflection index! 
        # IPA: ratio of the area under diastolic curve and the area under systolic curve
        # TspAsp: ratio between Tsp and Asp
        # AspdT: ratio between Asp and dT --> Stifness index
        # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
        TsysTdia = Tsys/Tdia
        TspTpi = Tsp/Tpi
        AdpAsp = Adp/Asp
        IPA = AUCdia/AUCsys
        TspAsp = Tsp/Asp
        AspdT = Asp/dT
        AspTpiTsp = Asp/(Tpi - Tsp)
        Rslope = Asp/Tsys

        # ---- Derivatives -----
        # Tu: time between the pulse onset and u
        # Tv: time between the pulse onset and v
        # Ta: time between the pulse onset and a
        # Tb: time between the pulse onset and b
        Tu = fiducials_diag["u"] - fiducials_diag["onset"]
        Tv = fiducials_diag["v"] - fiducials_diag["onset"]
        Ta = fiducials_diag["a"] - fiducials_diag["onset"]
        Tb = fiducials_diag["b"] - fiducials_diag["onset"]

        features[str(i)] = pd.DataFrame({"Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})

    with open(data_path + sub_ID + '/' + 'features_windows.pkl', 'wb') as f:
        pickle.dump(features, f)
        # features.to_csv(data_path + sub_ID + '/' + '/features_day1.csv', index = False)
        # fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials_day1.csv', index = False)

10_AC
0
1
2
3
4
5
6
7
12
13
47
48
99
100
101
102
111
112
113
153
154
155
166
168
169
170
171
172
173
174
175
176
177
178
181
182
184
185
186
250
294
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
477
524
529
538
539
540
541
549
555
569
570
572
584
589
590
592
593
594
11_EC
1
2
3
5
6
10
70
71
72
160
161
162
163
164
167
168
169
170
171
172
176
177
224
225
226
227
228
229
230
231
232
233
234
235
236
240

/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
489
490
491
493
494
13_MS
0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
17
18
217
262
263
264
265
266
267
268
269
270
271
272
273
274
307
308
309
310
311
312
313
314
315
316
317
318
319
321
322
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
355
358
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
412
414
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
14_AD
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
207
209
210
211
212
213
215
216
217
218
219
220
254
255
256

/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


260
261
262
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
400
401
402
403
404
405
406
407
408
410
411
412
413
414
415
416
417
418
419
420
526
17_EG
0
2
3
276
277
278
279
280
281
282
283
286
287
288
289
290
291
292
293
294
298
299
300
301
302
303
304
305
306
308
309
310
311
312
313
335
336
337
338
339
340
341
342
345
346
347
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
368
369
370
371
372
374
376
377
379
381
383
384
385
386
387
393
394
395
396
397
399
400
402
403
404
405
406
409
410
411
416
417
418
419


...

436
437
438
439
448
590
591
599
7_GM
212
235
236
242
243
247
248
249
250
251
252
253
254
255
273
276
277
278
280
281
282
284
285
286
287
288
289
290


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


291
292
293
294
295
296
297
298
299
300
307
308
309
310
311
312
313
321
322
323
324
335
337
338
339
349
350
351
352
362
369
370
374
375
376
377
378
379
380
381
399
400
401
402
439
8_MB
0
1
2
3
4
5
6
7
8
9
10
11
48
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
173
174
175
176
177
178
179
180
181
214
215
216
217
219
220
221
222
223
225
226
227
228
229
230
231
232
233
234
235
236
238
243
245
250
254
255
256
266
267
268
269
270
271
272
273
274
275
283
284
287
293
294
296
297
298
299
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
41

/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:53: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:67: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


272
273
274
275
282
283
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
322
323
324
325
326
327
329
330
331
333
334
335
336
337
338
342
345
350
351
354
369
370
373
377
381
382
383
384
385
386
394
395
396
398
399
400
401
402
403
405
406
407
408
409
410
411
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
447
448
449
450
451
452
453
454
455
456
457
458
460
464
465
466
468
469
470
471
472
473
474
475
476
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
494
495
496
497
559


In [ ]:
with 

In [14]:
with open(data_path + sub_ID + '/' + 'features_windows.pkl', 'wb') as f:
    print(f)
    pickle.dump(features, f)

<_io.BufferedWriter name='/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/10_AC/features_windows.pkl'>
